In [1]:
import pandas as pd
import numpy as np 
import datetime
import os
from treelib import Node, Tree
import pickle
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [8]:
data_test = pd.read_csv('E://天池竞赛/杭州地铁乘客流量/Metro_testA/testA_record_2019-01-28.csv')
data_submit = pd.read_csv('E://天池竞赛/杭州地铁乘客流量/Metro_testA/testA_submit_2019-01-29.csv')

In [148]:
in_test_pd = pd.DataFrame(columns=range(82))
out_test_pd = pd.DataFrame(columns=range(82))
start_time = data_test.time[1][:10] + ' 00:00'
#按每10分钟
tm_rang = pd.date_range(start_time,periods=6*24,freq='10T')

for i in range(len(tm_rang)):
    in_test_data = {}
    out_test_data = {}
    start_index  = datetime.datetime.fromtimestamp(tm_rang[i].value/1000000000 - 28800).strftime('%Y-%m-%d %H:%M:%S')
    if i != len(tm_rang)-1:
        end_index = datetime.datetime.fromtimestamp(tm_rang[i+1].value/1000000000  - 28800).strftime('%Y-%m-%d %H:%M:%S')
        sub_time_data = data_test[(data_test['time']>=start_index) & (data_test['time']<=end_index)]
    else:
        print(tm_rang[i])
        sub_time_data = data_test[(data_test['time']>=start_index)]
    for stationID, group in sub_time_data.groupby('stationID'):
        total_num = len(group)
        in_num = group[group.status==1].describe().status['count']
        out_num = total_num - in_num
        in_test_data[stationID] = in_num
        out_test_data[stationID] = out_num
    in_test_data['81'] = tm_rang[i]
    out_test_data['81'] = tm_rang[i]
    in_test_pd = in_test_pd.append(in_test_data, ignore_index=True)
    out_test_pd = out_test_pd.append(out_test_data, ignore_index=True)

d:\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexes\range.py:453: RuntimeWarning: unorderable types: int() < str(), sort order is undefined for incomparable objects
  return self._int64index.union(other)
d:\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
d:\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects


2019-01-28 23:50:00


In [149]:

pickle.dump(in_test_pd, open('in_test1_pd.pkl', 'wb'))
pickle.dump(out_test_pd, open('out_test1_pd.pkl', 'wb'))

In [96]:
in_num_pd = pd.DataFrame(columns=range(82))
out_num_pd = pd.DataFrame(columns=range(82))
rootdir = "E://天池竞赛/杭州地铁乘客流量/Metro_train/"
list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
#每个文件是一天的数据量
for i in range(0,len(list)):
    path = os.path.join(rootdir,list[i])
    if os.path.isfile(path):
        print(path)
        data_train_sample = pd.read_csv(path)
        #获取每天的起始时间
        start_time = data_train_sample.time[1][:10] + ' 00:00'
        #因为最终提交的是以10分钟为单位统计进出站人数，所以每天按10分钟等间隔划分，以此处理训练集
        tm_rang = pd.date_range(start_time,periods=6*24,freq='10T')
        for i in range(len(tm_rang)):
            
            in_data = {}
            out_data = {}
            #转换为'2019-01-01 00:00:00'的字符串形式
            start_index  = datetime.datetime.fromtimestamp(tm_rang[i].value/1000000000 - 28800).strftime('%Y-%m-%d %H:%M:%S')
            if i != len(tm_rang)-1:
                end_index = datetime.datetime.fromtimestamp(tm_rang[i+1].value/1000000000  - 28800).strftime('%Y-%m-%d %H:%M:%S')
                sub_time_data = data_train_sample[(data_train_sample['time']>=start_index) & (data_train_sample['time']<=end_index)]
            else:
                print(tm_rang[i])
                sub_time_data = data_train_sample[(data_train_sample['time']>=start_index)]
            #10分钟的时间段内所有站点的进出站总人数统计
            for stationID, group in sub_time_data.groupby('stationID'):
                total_num = len(group)
                in_num = group[group.status==1].describe().status['count']
                out_num = total_num - in_num
                in_data[stationID] = in_num
                out_data[stationID] = out_num
            in_data['81'] = tm_rang[i]
            out_data['81'] = tm_rang[i]
            in_num_pd = in_num_pd.append(in_data, ignore_index=True)
            out_num_pd = out_num_pd.append(out_data, ignore_index=True)

E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-01.csv


d:\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
d:\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects


2019-01-01 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-02.csv
2019-01-02 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-03.csv
2019-01-03 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-04.csv
2019-01-04 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-05.csv
2019-01-05 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-06.csv
2019-01-06 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-07.csv
2019-01-07 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-08.csv
2019-01-08 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-09.csv
2019-01-09 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-10.csv
2019-01-10 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-11.csv
2019-01-11 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-12.csv
2019-01-12 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-13.csv
2019-01-13 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01-14.csv
2019-01-14 23:50:00
E://天池竞赛/杭州地铁乘客流量/Metro_train/record_2019-01

In [97]:
#保存中间数据
pickle.dump(in_num_pd, open('in_num1_pd.pkl', 'wb'))
pickle.dump(out_num_pd, open('out_num1_pd.pkl', 'wb'))

In [12]:
in_num_pd = pickle.load(open('in_num1_pd.pkl', 'rb'))
out_num_pd = pickle.load(open('out_num1_pd.pkl', 'rb'))

In [ ]:
in_num_pd.fillna(0, inplace=True)
out_num_pd.fillna(0, inplace=True)
def predict(ds, y, future_periods = 144):
    prophet = Prophet(weekly_seasonality=True,  holidays=gaofen)
    train_df = pd.DataFrame({'ds':ds,
                             'y' :y})
    train_df['y'] = np.log(train_df['y']+ 0.001)
    #train_df['y'] = np.log(train_df['y']+ 0.001)
    #train_df['cap'] = train_df['y'].max()+0.5
    #train_df['floor'] = 0
    
    prophet.add_seasonality(name='weekly', period=7, fourier_order=3, prior_scale=1)
    prophet.fit(train_df)
    #print(group.entity_id.max() + ' ' + group.entity_id.max() +' is predicting...')
    
    future = prophet.make_future_dataframe(periods=future_periods, freq='10T')
    #future['cap']= train_df['y'].max()+0.5
    #future['floor'] = 0
    forecasts = prophet.predict(future)
    mae=np.sum(np.absolute( np.exp(forecasts['yhat'].head(len(train_df))) - train_df['y']))/len(train_df)
    #print(mae)
    y_pred = np.exp(forecasts['yhat'].tail(future_periods).values)
    #y_pred = np.exp(forecasts['yhat'])
    #prophet.plot(forecasts)
    #prophet.plot_components(forecasts)
    return y_pred

In [ ]:
in_pred_df = pd.DataFrame(columns=range(81))
for i in range(81):
    pred = predict( in_num_pd['81'], in_num_pd[i], 4*144)
    in_pred_df[i] = pred
out_pred_df = pd.DataFrame(columns=range(81))
for i in range(81):
    pred = predict( out_num_pd['81'], out_num_pd[i], 4*144)
    out_pred_df[i] = pred

In [10]:
pickle.dump(in_pred_df, open('in_pred_pd_log.pkl', 'wb'))
pickle.dump(out_pred_df, open('out_pred_pd_log.pkl', 'wb'))

NameError: name 'in_pred_df' is not defined

In [ ]:
#MAE计算
mae = np.sum(np.absolute(forecast['yhat'].head(len(train_df))-train_df['y']))/len(train_df)

In [ ]:
#广度搜索
def BFS(node0):
    #queue本质上是堆栈，用来存放需要进行遍历的数据
    #order里面存放的是具体的访问路径
    queue,order = [],[]
    #首先将初始遍历的节点放到queue中，表示将要从这个点开始遍历
    # 由于是广度优先，也就是先访问初始节点的所有的子节点，所以可以
    queue.append(node0)
    order.append(node0)
    while queue:
        #queue.pop(0)意味着是队列的方式出元素，就是先进先出，而下面的for循环将节点v的所有子节点
        #放到queue中，所以queue.pop(0)就实现了每次访问都是先将元素的子节点访问完毕，而不是优先叶子节点
        v = queue.pop(0)
        for w in self.sequense[v]:
            if w not in order:
                # 这里可以直接order.append(w) 因为广度优先就是先访问节点的所有下级子节点，所以可以
                # 将self.sequense[v]的值直接全部先给到order
                order.append(w)
                queue.append(w)
    return order

In [4]:
#构建站点树，以某个站点为根节点，广度搜索遍历图，构造多叉树
class stationTree():
    def __init__(self,start_station, maxDeep, mapArray):
        self.stationTree = Tree()
        self.total = []
        self.total.append(start_station)
        self.start_station = start_station
        self.maxDeep = maxDeep
        self.mapArray = mapArray
        
    def creat_tree_by_BFS(self):
        queue = []
        queue.append(self.start_station)
        self.stationTree.create_node(self.start_station, self.start_station)
        while queue and self.stationTree.depth()<self.maxDeep:
            v = queue.pop(0)
            for i in range(len(self.mapArray)):
                if self.mapArray[v][i] == 1 and i not in self.total:
                    queue.append(i)
                    self.total.append(i)
                    self.stationTree.create_node(i, i, parent=v)
    def creat_train_data(self, in_df, out_df, time_point):
        train_x = []
        train_y = []
        for day in range(25):
            feature_row = []
            now_index = day*144+time_point 
            for s_id in self.total[1:]:
                forward_index = now_index - int(self.stationTree.level(s_id)/2)
                #print(forward_index)
                if forward_index >= day*144+33:
                    feature_row.append(in_df.iloc[forward_index, s_id])
            train_x.append(feature_row)
            train_y.append(out_df.iloc[now_index, self.start_station])
        return train_data, train_y
    

In [5]:
roadMap = pd.read_csv('E://天池竞赛/杭州地铁乘客流量/Metro_roadMap.csv', index_col=0)
mapArray = roadMap.values
start_id = 4
s_tree = stationTree(start_id, 25, mapArray)
s_tree.creat_tree_by_BFS()
print(s_tree.stationTree.size())
s_tree.stationTree.show()

81
4
├── 3
│   └── 2
│       └── 1
│           └── 0
└── 5
    ├── 6
    │   └── 7
    │       └── 8
    │           └── 9
    │               ├── 10
    │               │   ├── 11
    │               │   │   └── 12
    │               │   │       └── 13
    │               │   │           └── 14
    │               │   ├── 50
    │               │   │   └── 49
    │               │   └── 52
    │               │       └── 53
    │               │           └── 54
    │               │               └── 55
    │               │                   └── 56
    │               │                       └── 57
    │               │                           └── 58
    │               │                               └── 59
    │               │                                   └── 60
    │               │                                       └── 61
    │               │                                           └── 62
    │               │                                               └── 63


In [189]:
#7:00 ->42
train_data_42, train_y_42 = s_tree.creat_train_data(in_num_pd,out_num_pd, 42)
train_data_42_df = pd.DataFrame(train_data_42)

In [184]:
train_data_42_df

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
0,56.0,41.0,51.0,26.0,4.0,15.0,29.0,95.0,30.0,4.0,...,0.0,6.0,14.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0
1,188.0,93.0,128.0,77.0,25.0,14.0,77.0,129.0,62.0,4.0,...,2.0,14.0,6.0,0.0,0.0,0.0,1.0,4.0,4.0,0.0
2,167.0,99.0,161.0,77.0,24.0,22.0,88.0,177.0,68.0,9.0,...,0.0,8.0,2.0,0.0,0.0,0.0,1.0,1.0,3.0,0.0
3,147.0,97.0,122.0,66.0,19.0,18.0,67.0,76.0,56.0,5.0,...,3.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
4,70.0,44.0,73.0,37.0,7.0,3.0,43.0,56.0,25.0,4.0,...,1.0,7.0,4.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0
5,53.0,43.0,58.0,21.0,5.0,8.0,42.0,103.0,18.0,0.0,...,1.0,6.0,8.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0
6,165.0,111.0,153.0,81.0,22.0,16.0,74.0,158.0,54.0,4.0,...,5.0,13.0,9.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0
7,167.0,107.0,148.0,93.0,20.0,15.0,66.0,103.0,55.0,3.0,...,5.0,10.0,7.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0
8,137.0,87.0,161.0,70.0,25.0,18.0,70.0,93.0,51.0,5.0,...,2.0,12.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,156.0,101.0,132.0,64.0,21.0,25.0,76.0,80.0,53.0,2.0,...,1.0,7.0,4.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [191]:
lr = LinearRegression()
lr.fit(train_data_42,train_y_42)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [143]:
y_hat = lr.predict([train_data[5]])

In [144]:
y_hat

array([44.])

In [145]:
train_y[5]

44.0

In [152]:
in_test_pd.fillna(0, inplace=True)
in_test_pd.fillna(0, inplace=True)

In [6]:
feature_row = []
test_y = []
now_index = 42
for s_id in s_tree.total[1:]:
    forward_index = now_index - int(s_tree.stationTree.level(s_id)/2)
    print(forward_index)
    if forward_index >= 33:
        feature_row.append(in_test_pd.iloc[forward_index, s_id])
test_y.append(out_test_pd.iloc[now_index, 4])

42
42
41
41
41
41
41
41
41
41
40
40
40
40
40
40
40
39
39
39
39
39
39
39
39
39
39
39
39
39
38
38
38
38
38
38
38
38
38
38
38
37
37
37
37
37
37
37
36
36
36
36
36
36
35
35
35
35
35
35
35
34
34
34
34
34
34
34
34
33
33
33
33
33
33
32
32
32
32
32


In [159]:
y_hat = lr.predict([feature_row])
y_hat

array([112.0288367])

In [7]:
test_y

[100.0]

In [2]:
in_test_pd = pickle.load(open('in_test1_pd.pkl', 'rb'))
out_test_pd = pickle.load(open('out_test1_pd.pkl', 'rb'))

In [34]:
submit_data = pd.read_csv("E://天池竞赛/杭州地铁乘客流量/Metro_testA/testA_submit_2019-01-29.csv")

In [8]:
submit_data

,stationID,startTime,endTime,inNums,outNums
0,0,2019-01-29 00:00:00,2019-01-29 00:10:00,NaN,NaN
1,0,2019-01-29 00:10:00,2019-01-29 00:20:00,NaN,NaN
2,0,2019-01-29 00:20:00,2019-01-29 00:30:00,NaN,NaN
3,0,2019-01-29 00:30:00,2019-01-29 00:40:00,NaN,NaN
4,0,2019-01-29 00:40:00,2019-01-29 00:50:00,NaN,NaN
5,0,2019-01-29 00:50:00,2019-01-29 01:00:00,NaN,NaN
6,0,2019-01-29 01:00:00,2019-01-29 01:10:00,NaN,NaN
7,0,2019-01-29 01:10:00,2019-01-29 01:20:00,NaN,NaN
8,0,2019-01-29 01:20:00,2019-01-29 01:30:00,NaN,NaN
9,0,2019-01-29 01:30:00,2019-01-29 01:40:00,NaN,NaN


In [29]:
in_pred = pickle.load(open('in_pred_pd_log.pkl','rb'))
out_pred = pickle.load(open('out_pred_pd_log.pkl','rb'))

In [30]:
pred_29_in = in_pred.tail(144)
pred_29_out = out_pred.tail(144)

In [31]:
pred_29_in_arr = [i for item in pred_29_in.values.astype(np.int).T for i in item]
pred_29_out_arr = [i for item in pred_29_out.values.astype(np.int).T for i in item]

In [32]:
pred_29_in_arr = pred_29_in.values.astype(np.int).T.reshape(1,-1)
pred_29_out_arr = pred_29_out.values.astype(np.int).T.reshape(1,-1)

In [140]:
len(pred_29_in_arr)

1

In [148]:
pred_29_in.values.astype(np.int).T[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   1,   1,   3,   5,   9,  16,
        27,  45,  73, 113, 169, 242, 331, 431, 536, 633, 713, 765, 783,
       769, 726, 662, 586, 506, 429, 359, 298, 246, 205, 172, 146, 126,
       111, 100,  92,  87,  84,  84,  85,  89,  94, 101, 110, 120, 133,
       147, 161, 176, 190, 204, 214, 222, 226, 226, 222, 215, 204, 191,
       178, 163, 149, 136, 124, 114, 105,  98,  92,  88,  86,  84,  84,
        86,  89,  93,  98, 105, 112, 120, 129, 137, 144, 149, 152, 152,
       148, 140, 129, 115,  99,  83,  67,  52,  40,  29,  21,  15,  10,
         7,   4,   3,   2,   1,   0,   0,   0,   0,   0,   0,   0,   0,
         0])

In [35]:
submit_data['inNums'] = pred_29_in_arr[0]
submit_data['outNums'] = pred_29_out_arr[0]

In [36]:
submit_data.head()

,stationID,startTime,endTime,inNums,outNums
0,0,2019-01-29 00:00:00,2019-01-29 00:10:00,0,0
1,0,2019-01-29 00:10:00,2019-01-29 00:20:00,0,0
2,0,2019-01-29 00:20:00,2019-01-29 00:30:00,0,0
3,0,2019-01-29 00:30:00,2019-01-29 00:40:00,0,0
4,0,2019-01-29 00:40:00,2019-01-29 00:50:00,0,0


In [30]:
submit_data

,stationID,startTime,endTime,inNums,outNums
0,0,2019-01-29 00:00:00,2019-01-29 00:10:00,0,0
1,0,2019-01-29 00:10:00,2019-01-29 00:20:00,0,0
2,0,2019-01-29 00:20:00,2019-01-29 00:30:00,0,0
3,0,2019-01-29 00:30:00,2019-01-29 00:40:00,0,0
4,0,2019-01-29 00:40:00,2019-01-29 00:50:00,0,0
5,0,2019-01-29 00:50:00,2019-01-29 01:00:00,0,0
6,0,2019-01-29 01:00:00,2019-01-29 01:10:00,0,0
7,0,2019-01-29 01:10:00,2019-01-29 01:20:00,0,0
8,0,2019-01-29 01:20:00,2019-01-29 01:30:00,0,0
9,0,2019-01-29 01:30:00,2019-01-29 01:40:00,0,0


In [37]:
submit_data.iloc[30:43]

,stationID,startTime,endTime,inNums,outNums
30,0,2019-01-29 05:00:00,2019-01-29 05:10:00,0,0
31,0,2019-01-29 05:10:00,2019-01-29 05:20:00,0,0
32,0,2019-01-29 05:20:00,2019-01-29 05:30:00,0,0
33,0,2019-01-29 05:30:00,2019-01-29 05:40:00,0,0
34,0,2019-01-29 05:40:00,2019-01-29 05:50:00,0,0
35,0,2019-01-29 05:50:00,2019-01-29 06:00:00,1,0
36,0,2019-01-29 06:00:00,2019-01-29 06:10:00,2,0
37,0,2019-01-29 06:10:00,2019-01-29 06:20:00,4,0
38,0,2019-01-29 06:20:00,2019-01-29 06:30:00,7,0
39,0,2019-01-29 06:30:00,2019-01-29 06:40:00,14,0


In [134]:

a = np.array([[2,3], [2,3],[1,1]])
print(a)
print(a.reshape(1, -1))

[[2 3]
 [2 3]
 [1 1]]
[[2 3 2 3 1 1]]


In [38]:
submit_data.to_csv('result-3-25.csv',encoding='utf-8',index=False)